In [1]:
import os

In [2]:
import numpy as np

files = os.listdir("/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/")
for i,f in enumerate(files):
    files[i] = "/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/"+f

In [3]:
from pydub import AudioSegment

audio = AudioSegment.from_file(files[0])

In [6]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


In [10]:
samples = np.asarray(frames)

In [14]:
import numpy as np
rec = AudioSegment(np.asarray(frames),
                      frame_rate=RATE,
                      sample_width=p.get_sample_size(FORMAT),
                      channels=1)
rec.export("/home/andrea/Scrivania/rec.wav", format="wav")

ValueError: data length must be a multiple of '(sample_width * channels)'

In [13]:
p.get_sample_size(FORMAT)

2

In [ ]:
wf = wave.open("/home/andrea/Scrivania/rec.wav", "wb")
# set the channels
wf.setnchannels(CHANNELS)
# set the sample format
wf.setsampwidth(p.get_sample_size(FORMAT))
# set the sample rate
wf.setframerate(RATE)
# write the frames as bytes
wf.writeframes(b"".join(frames))
# close the file
wf.close()

In [19]:
audio_rec = np.frombuffer(samples)

MODEL TEST

In [21]:
from tensorflow import keras

model = keras.models.load_model("audio_model_bidirectional_LSTM.h5")

dataset = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/dataset.npy")
std_dev = np.std(dataset)
mean = np.mean(dataset)
maximum = np.max(dataset)
min = np.min(dataset)
scale = max(maximum, abs(min))

def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [60]:
frames = []
do_prediction = False
window = np.zeros((1,16,16516))
filled = 0
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=16384)
add = False
while True:
    data = stream.read(16384)
    frames.append(data)
    if len(np.frombuffer(np.asarray(frames))) == 16384:
        s_arr = np.zeros((1,1,16516))
        s_arr[0,0,:16384] = np.frombuffer(np.asarray(frames))
        frames = []
        add = True
        print("Full frame")
    if filled <= 15 and add:
        window[0,filled,:] = normalize(s_arr[0,0,:], std_dev, mean, scale)
        filled += 1

    elif filled > 15 and add:
        window = np.delete(window, 0, 1)
        window = np.hstack((window, s_arr))
        print("Full Window")
        do_prediction = True
    add = False
    if do_prediction:
        c = model.predict_classes(window)
        do_prediction = False
        print(c)

Full frame


/home/andrea/Scrivania/RC_Carta/venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


KeyboardInterrupt: 

In [58]:
stream.stop_stream()
stream.close()
p.terminate()

In [47]:
t_base = np.zeros((1,15,64))
t_add = np.ones((1,1,64,))

In [48]:
merged = np.hstack((t_base,t_add))